In [ ]:
from google.colab import drive
drive.mount("/drive")

Mounted at /drive


In [ ]:

import os, json, math, random, numpy as np, tensorflow as tf
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix
tf.__version__

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

# Paths
ROOT = Path("/drive/MyDrive")
DATA_ROOT = ROOT / "Tumour"     # expects subfolders: train/ valid/ test
ARTIFACTS = ROOT / "artifacts"
ARTIFACTS.mkdir(parents=True, exist_ok=True)

# Hyperparams (kept aligned with your original)
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 15
PATIENCE = 4
USE_CLASS_WEIGHTS = True
FINE_TUNE_LAST_N_LAYERS = 50

AUTOTUNE = tf.data.AUTOTUNE

# %% [data] Load splits from directories
train_dir = DATA_ROOT / "train"
val_dir   = DATA_ROOT / "valid"
test_dir  = DATA_ROOT / "test"

def make_ds(dir_path, shuffle, batch_size=BATCH_SIZE):
    return tf.keras.utils.image_dataset_from_directory(
        directory=str(dir_path),
        labels="inferred",
        label_mode="int",
        image_size=(IMG_SIZE, IMG_SIZE),
        interpolation="bilinear",
        shuffle=shuffle,
        seed=SEED,
        batch_size=batch_size
    )

train_ds = make_ds(train_dir, shuffle=True)
val_ds   = make_ds(val_dir,   shuffle=False)
test_ds  = make_ds(test_dir,  shuffle=False)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

# Persist class map (index -> name), same as your original
with open(ARTIFACTS / "class_indices.json", "w") as f:
    json.dump({i: c for i, c in enumerate(class_names)}, f, indent=2)
print("Saved:", ARTIFACTS / "class_indices.json")

# Prefetch for speed
def tune(ds, training=False):
    if training:
        ds = ds.shuffle(1000, seed=SEED, reshuffle_each_iteration=True)
    return ds.prefetch(AUTOTUNE)

train_ds = tune(train_ds, training=True)
val_ds   = tune(val_ds)
test_ds  = tune(test_ds)

# %% [preprocessing & augmentation]
from tensorflow.keras import layers, models

# Data augmentation (roughly mirrors: HFlip, Rotation(10°), RandomResizedCrop scale(0.9,1.0))
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    # Zoom-in only (≈ crop then resize back up)
    layers.RandomZoom(height_factor=(-0.1, 0.0), width_factor=(-0.1, 0.0)),
])

# ImageNet mean/std normalization (matches your torchvision.Normalize) for the CUSTOM CNN
# Normalization takes variance (std^2)
imagenet_norm = layers.Normalization(
    mean=[0.485, 0.456, 0.406],
    variance=[0.229**2, 0.224**2, 0.225**2],
    name="imagenet_norm"
)

# EfficientNetB0 in Keras has built-in rescaling & normalization inside the model,
# so we DO NOT apply the above mean/std for the EfficientNet branch. We still rescale for the custom CNN.

# %% [class weights]
def compute_class_weights(dataset, num_classes):
    counts = np.zeros(num_classes, dtype=np.int64)
    for _, y in dataset.unbatch():
        counts[int(y.numpy())] += 1
    # Same idea: inverse frequency, normalized to average weight=1
    w = 1.0 / np.clip(counts, 1, None)
    w = w / w.sum() * num_classes
    return {i: float(w[i]) for i in range(num_classes)}, counts

class_weights, counts = (None, None)
if USE_CLASS_WEIGHTS:
    class_weights, counts = compute_class_weights(train_ds, num_classes)
    print("Class counts:", counts)
    print("Class weights:", class_weights)


def build_custom_cnn(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = layers.Rescaling(1./255)(inputs)
    x = data_augmentation(x)
    x = imagenet_norm(x)  # match PyTorch normalization

    # Conv blocks
    def block(x, out_ch):
        x = layers.Conv2D(out_ch, 3, padding="same", use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.MaxPooling2D()(x)
        return x

    x = block(x, 32)
    x = block(x, 64)
    x = block(x, 128)
    x = block(x, 256)

    # Extra conv + GAP
    x = layers.Conv2D(256, 3, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.GlobalAveragePooling2D()(x)

    # Classifier
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs, outputs, name="CustomCNN")
    return model

# EfficientNetB0 (frozen / fine-tune)
def build_efficientnet_b0(num_classes, train_base=False, fine_tune_last_n=0, name="EffNetB0"):
    # NOTE: EfficientNetB0 in Keras includes a rescaling layer internally
    base = tf.keras.applications.EfficientNetB0(
        include_top=False, weights="imagenet", pooling="avg", input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    base.trainable = train_base
    if train_base and fine_tune_last_n > 0:
        # Freeze all but the last N layers of the base model
        for layer in base.layers[:-fine_tune_last_n]:
            layer.trainable = False

    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    # We still apply augmentation here; EfficientNet will handle its own normalization internally.
    x = data_augmentation(inputs)
    x = base(x, training=False)  # important for BatchNorm behavior during fine-tune warmup
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    model = models.Model(inputs, outputs, name=name)
    return model


def count_trainable_params(model):
    return int(np.sum([np.prod(v.shape) for v in model.trainable_weights]))

def compile_model(model, lr=1e-3, wd=1e-4):
    # AdamW is available in TF 2.11+; fallback to Adam if needed
    try:
        opt = tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=wd)
    except Exception:
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(
        optimizer=opt,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

def train_model(name, model, train_ds, val_ds, test_ds, lr=1e-3, epochs=EPOCHS, patience=PATIENCE,
                class_weights=None):
    compile_model(model, lr=lr, wd=1e-4)
    print(f"[{name}] Trainable params: {count_trainable_params(model):,}")

    ckpt_path = ARTIFACTS / f"best_{name}.keras"
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=patience, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.ModelCheckpoint(filepath=str(ckpt_path), monitor="val_loss", save_best_only=True, verbose=1)
    ]

    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        class_weight=class_weights,
        verbose=2,
        callbacks=callbacks
    )

    te_loss, te_acc = model.evaluate(test_ds, verbose=0)
    print(f"[{name}] TEST   loss={te_loss:.4f} acc={te_acc:.4f}")

    # Predictions for full report
    y_true = []
    y_prob = []
    for x_batch, y_batch in test_ds:
        y_true.extend(y_batch.numpy().tolist())
        y_prob.extend(model.predict(x_batch, verbose=0).tolist())
    y_true = np.array(y_true)
    y_prob = np.array(y_prob)
    y_pred = np.argmax(y_prob, axis=1)

    print(f"[{name}] REPORT\n" + classification_report(y_true, y_pred, target_names=class_names, digits=4))
    print(f"[{name}] CONFUSION\n{confusion_matrix(y_true, y_pred)}")

    return te_acc, model, str(ckpt_path)

# 1) Custom CNN
custom = build_custom_cnn(num_classes)
acc_custom, model_custom, ckpt_custom = train_model(
    "CustomCNN", custom, train_ds, val_ds, test_ds, lr=1e-3, class_weights=class_weights
)

# 2) EfficientNetB0 (frozen base)
eff_frozen = build_efficientnet_b0(num_classes, train_base=False, name="EffNetB0_Frozen")
acc_frozen, model_frozen, ckpt_frozen = train_model(
    "EffNetB0_Frozen", eff_frozen, train_ds, val_ds, test_ds, lr=1e-3, class_weights=class_weights
)

# 3) EfficientNetB0 (fine-tune last N layers)
eff_ft = build_efficientnet_b0(num_classes, train_base=True, fine_tune_last_n=FINE_TUNE_LAST_N_LAYERS, name="EffNetB0_FT")
acc_ft, model_ft, ckpt_ft = train_model(
    "EffNetB0_FT", eff_ft, train_ds, val_ds, test_ds, lr=1e-4, class_weights=class_weights
)

# Pick best by test accuracy
scores = [
    (acc_custom, model_custom, "custom_cnn", ckpt_custom),
    (acc_frozen, model_frozen, "efficientnet_b0", ckpt_frozen),
    (acc_ft,     model_ft,     "efficientnet_b0", ckpt_ft),
]
best_acc, best_model, arch, best_ckpt = max(scores, key=lambda t: t[0])
print(f"[BEST] acc={best_acc:.4f} arch={arch}")

# Save a unified "best" export and the class map path (parallel to your .pt dict)
best_export_path = ARTIFACTS / "best_model"
best_model.save(best_export_path, include_optimizer=False)
meta = {
    "model_name": "BEST",
    "arch": arch,
    "num_classes": num_classes,
    "class_map_path": str(ARTIFACTS / "class_indices.json"),
    "keras_checkpoint": best_ckpt
}
with open(ARTIFACTS / "best_model_meta.json", "w") as f:
    json.dump(meta, f, indent=2)
print("Saved ->", best_export_path)
print("Saved ->", ARTIFACTS / "best_model_meta.json")

# %% [inference helper] Single-image predict (mirrors your predict_image)
from PIL import Image

def load_best(ckpt_dir=ARTIFACTS / "best_model"):
    model = tf.keras.models.load_model(ckpt_dir)
    with open(ARTIFACTS / "class_indices.json", "r") as f:
        idx2cls = {int(k): v for k, v in json.load(f).items()}
    return model, idx2cls

def predict_image(img_path, model, idx2cls):
    # The model already contains preprocessing/augmentation layers as defined above
    img = tf.keras.utils.load_img(str(img_path), target_size=(IMG_SIZE, IMG_SIZE))
    x = tf.keras.utils.img_to_array(img)
    x = tf.expand_dims(x, axis=0)  # (1, H, W, 3)
    probs = model.predict(x, verbose=0)[0]
    pred_idx = int(np.argmax(probs))
    return idx2cls[pred_idx], {idx2cls[i]: float(probs[i]) for i in range(len(probs))}

# Example usage (optional; ensure you have at least one test image):
# sample_class = class_names[0]
# sample_path = next((test_dir / sample_class).glob("*.jpg"))
# model_loaded, idx2cls = load_best()
# pred, probs = predict_image(sample_path, model_loaded, idx2cls)
# print("Pred:", pred)
# print("Probs:", probs)

# %% [optional] Copy artifacts to Drive (uncomment in Colab)
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# !mkdir -p "/content/drive/MyDrive/BrainTumorArtifacts_TF"
# !cp -v /content/artifacts/* "/content/drive/MyDrive/BrainTumorArtifacts_TF/"
# print("Artifacts copied to /content/drive/MyDrive/BrainTumorArtifacts_TF")


Found 1695 files belonging to 4 classes.
Found 502 files belonging to 4 classes.
Found 246 files belonging to 4 classes.
Classes: ['glioma', 'meningioma', 'no_tumor', 'pituitary']
Saved: /drive/MyDrive/artifacts/class_indices.json
Class counts: [564 358 335 438]
Class weights: {0: 0.7211531115429481, 1: 1.1361183098050915, 2: 1.2141204624185755, 3: 0.9286081162333852}
[CustomCNN] Trainable params: 1,012,644
Epoch 1/15

Epoch 1: val_loss improved from inf to 5.50596, saving model to /drive/MyDrive/artifacts/best_CustomCNN.keras
53/53 - 453s - 9s/step - accuracy: 0.6708 - loss: 0.8314 - val_accuracy: 0.3625 - val_loss: 5.5060 - learning_rate: 1.0000e-03
Epoch 2/15


KeyboardInterrupt: 